## Implementing binary decision trees with real valued features
**Multi class supported - Classes IDs are 0 through C**

In [1]:
from sklearn.datasets import make_classification
from sklearn.cross_validation import train_test_split
import numpy as np
from __future__ import division

### Generate a fake dataset 

In [2]:
#artificial training dataset
Xtrain = np.random.rand(1000,2)
y = np.ones(1000).astype(int)
y[(Xtrain[:,0] < 0.25)] = 0
y[(Xtrain[:,1] > 0.55)] = 2
np.unique(y)

array([0, 1, 2])

### Another fake dataset with more features

In [506]:
X,y = make_classification(n_samples=2000,n_features=10,n_informative=5,n_classes=2)
print X.shape
print np.unique(y) #original labels are 0 and 1
ind = y==0
y[ind] = -1
print np.unique(y)

(2000, 10)
[0 1]
[-1  1]


# Decision tree implementation

### Function to count number of mistakes while predicting majority class

Recall from the lecture that prediction at an intermediate node works by predicting the **majority class** for all data points that belong to this node.

Now, we will write a function that calculates the number of **missclassified examples** when predicting the **majority class**. This will be used to help determine which feature is the best to split on at a given node of the tree.

**Note**: Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node. 

** Steps to follow **:
* ** Step 1:** Calculate the number of +1 and -1
* ** Step 2:** Since we are assuming majority class prediction, all the data points that are **not** in the majority class are considered **mistakes**.
* ** Step 3:** Return the number of **mistakes**.

Now, let us write the function `intermediate_node_num_mistakes` which computes the number of misclassified examples of an intermediate node given the set of labels (y values) of the data points contained in the node. 

In [3]:
def intermediate_node_num_mistakes(labels_in_node):
    
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    C,unique_counts = np.unique(labels_in_node,return_counts=True) #the id of classes and number of each
    
    return (len(labels_in_node) - unique_counts[np.argmax(unique_counts)])    

In [4]:
def reached_minimum_node_size(y, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    if y.shape[0] <= min_node_size:
        print y.shape[0]
        return True

## Function to pick best feature to split on

The function **best_splitting_feature** takes 2 arguments: 
1. The feature matrix X [N datapoints x p features]
2. The vector of labels [N x 1]

Recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

* The second implementation uses the Information Gain to find the optimal split and bins the data into 10 bins

In [13]:
# X matrix of features (p datapoints x N features)
# y vector of labels (p x 1)

def best_splitting_feature(X, y):
        
    Nbins = 10  #the number of bins to split on
    best_feature = None # Keep track of the best feature 
    best_threshold = None
    best_I = -1     # Keep track of the best info gain so far 

    #the number of data points in the parent node
    num_data_points = y.shape[0]
    
    # Loop through each feature to consider splitting on that feature
    for feature in range(X.shape[1]):
        
        fvals = X[:,feature]
        fvals = np.sort(fvals)  #sorting the values
        if num_data_points > Nbins:            
            fvals = fvals[range(0,num_data_points,Nbins)]
        
        #loop through all values of current feature to find the best split
        for threshold in fvals:

            # The left split will have all data points where the feature value is smaller than threshold
            ind_left = X[:,feature] < threshold
            left_split = X[ind_left,feature]
             # The right split will have all data points where the feature value is larger or equal
            ind_right = X[:,feature] >= threshold
            right_split = X[ind_right,feature]
            
            #compute info-gain for current feature and threshold split
            I = infogain(y,y[ind_left],y[ind_right])
            
            # If this is the best error we have found so far, store the feature as best_feature
            # the threshold as the best threshold and the error as best_error
            if I > best_I:
                best_feature = feature
                best_threshold = threshold
                best_I = I
        
    return best_feature, best_threshold # Return the best feature and threshold

In [6]:
def infogain(yparent,yleft,yright):
    
    Nparent = len(yparent)
    Nleft = len(yleft)
    Nright = len(yright)
    
    #when one of the splits is empty returns I = 0
    if Nleft ==0 or Nright == 0:
        I = 0
    else:
        #compute information gain
        I = entropy(yparent) -( (Nleft/Nparent)*entropy(yleft) + (Nright/Nparent)*entropy(yright) )   

    return I


In [7]:
#entropy for multiple classes
def entropy(y):
    C,unique_counts = np.unique(y,return_counts=True) #the id of classes and number of each
    Pc = unique_counts/len(y)
    H = -(Pc*np.log(Pc)).sum()
    return H    

## Building the tree

With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'splitting_feature'  : The feature that this node splits on.
    }

First, we will write a function that creates a leaf node given a set of target values. Fill in the places where you find `## YOUR CODE HERE`. There are **three** places in this function for you to fill in.

In [8]:
def create_leaf(target_values,C):

    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True,
            'prediction': None,
            'labels_distribution':None                       }   
    
    # Count the number of data points of each class in the leaf.
    C_in_node,unique_counts = np.unique(target_values,return_counts=True) #the id of classes and number of each
    leaf['prediction'] = C_in_node[np.argmax(unique_counts)]
    
    Classes = np.zeros(C)
    Classes[C_in_node] = unique_counts/len(target_values)
    leaf['labels_distribution'] = Classes
    
    # Return the leaf node        
    return leaf 

In [130]:
def decision_tree_create(X, y, N_features_to_sample, current_depth = 0, max_depth = 10, C=-1):

    #parameter to set the minimum size of a node
    min_node_size = 5
    
    #determine # of classes in root if the argument is not provided
    if current_depth == 0 and C == -1:
        C = len(np.unique(y))
        
    #randomly sample a subset of features
    Nfeatures = X.shape[1]
    features = np.random.choice(Nfeatures, N_features_to_sample, replace=False)    
    
    #select only the features sampled for this run
    Xcurrent = X[:,features]
    target_values = y
    
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    print "Features selected = %s" % features
    

    # Stopping condition 1
    # (Check if there are mistakes at current node, i.e. if the node is pure.)
    if intermediate_node_num_mistakes(target_values) == 0:  
        print "No Mistakes at current node - Stopping."     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values,C)
    
    #Stopping condition 2: min node size reached
    if reached_minimum_node_size(y, min_node_size):
        print "Minimum node size reached - Stopping"
        return create_leaf(y,C)
    
    # Stopping condition 3: (limit tree depth)
    if current_depth >= max_depth:  
        print "Reached maximum depth. Stopping for now."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values,C)

    # Find the best splitting feature and its threshold
    splitting_feature,splitting_thres = best_splitting_feature(Xcurrent,y)
    splitting_feature = features[splitting_feature]
    
    # Split on the best feature that we found. 
    ind_left = X[:,splitting_feature] < splitting_thres
    left_split = X[ind_left,:]
    y_left = y[ind_left]

    ind_right = X[:,splitting_feature] >= splitting_thres
    right_split = X[ind_right,:]
    y_right = y[ind_right]
            
    #print y_left.shape, y_right.shape
    print "Split on feature %s. (%s, %s), Threshold = %s" % (\
                      splitting_feature, y_left.shape, y_right.shape, splitting_thres)
    
    # Create a leaf node if the split is "perfect"
    if y_left.shape == y.shape[0]:
        print "Creating leaf node."
        return create_leaf(y_left,C)
    if y_right.shape == y.shape[0]:
        print "Creating leaf node."
        return create_leaf(y_right,C)
        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, y_left, N_features_to_sample, current_depth + 1, max_depth, C)        
    right_tree = decision_tree_create(right_split, y_right, N_features_to_sample, current_depth + 1, max_depth, C)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'threshold'        : splitting_thres,
            'left'             : left_tree, 
            'right'            : right_tree,
            'labels_distribution': None 
            
            }

Here is a recursive function to count the nodes and leaves in your tree:

In [15]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [16]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1 
    return count_leaves(tree['left']) + count_leaves(tree['right'])
        

## Build the tree!

Now that all the tests are passing, we will train a tree model on the **train_data**. Limit the depth to 6 (**max_depth = 6**) to make sure the algorithm doesn't run for too long. Call this tree **my_decision_tree**. 

In [17]:
tree1 = decision_tree_create(Xtrain,y,Xtrain.shape[1],max_depth=2)

--------------------------------------------------------------------
Subtree, depth = 0 (1000 data points).
Features selected = [0 1]
Split on feature 1. ((540,), (460,)), Threshold = 0.549823963967
--------------------------------------------------------------------
Subtree, depth = 1 (540 data points).
Features selected = [0 1]
Split on feature 0. ((130,), (410,)), Threshold = 0.262277699549
--------------------------------------------------------------------
Subtree, depth = 2 (130 data points).
Features selected = [0 1]
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (410 data points).
Features selected = [1 0]
No Mistakes at current node - Stopping.
--------------------------------------------------------------------
Subtree, depth = 1 (460 data points).
Features selected = [0 1]
Split on feature 1. ((10,), (450,)), Threshold = 0.559855977152
-----------------------------------------------------------

In [18]:
count_nodes(tree1)

7

In [19]:
count_leaves(tree1)

4

## Making predictions with a decision tree

As discussed in the lecture, we can make predictions from the decision tree with a simple recursive function. Below, we call this function `classify`, which takes in a learned `tree` and a test point `x` to classify.  We include an option `annotate` that describes the prediction path when set to `True`.

In [20]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # split on feature.
        val_split_feature = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], tree['threshold'])
        if val_split_feature < tree['threshold']:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'],x, annotate)

In [23]:
print y[1]
classify(tree1,Xtrain[1,:],annotate=True)


1
Split on 1 = 0.549823963967
Split on 0 = 0.262277699549
At leaf, predicting 1


1

In [40]:
tree1['right']['left']

{'is_leaf': True,
 'labels_distribution': array([ 0.1,  0. ,  0.9]),
 'left': None,
 'prediction': 2,
 'right': None,
 'splitting_feature': None}

#### compute the path to the leaf followed by data point x  

In [26]:
def datapath(tree, x, branch = 1):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        return branch 
    else:
        # split on feature.
        split_feature = tree['splitting_feature']
        split_threshold = tree['threshold']

        if x[split_feature] < split_threshold:
            return datapath(tree['left'], x, 2*branch)
        else:
            return datapath(tree['right'],x, 2*branch+1)

In [29]:
datapath(tree1,Xtrain[0])

7

In [49]:
del(tree1['right']['right']['threshold'])

In [50]:
tree1['right']['right']

{'is_leaf': True,
 'labels_distribution': array([ 0.,  0.,  1.]),
 'left': None,
 'prediction': 2,
 'right': None,
 'splitting_feature': None}

In [27]:
def evaluate_classification_error(tree, X, y):
    # Apply the classify(tree, x) to each row in your data
    prediction = map(lambda x: classify(tree,x), X)
    # Once you've made the predictions, calculate the classification error and return it
    mistakes = sum(prediction != y)
    error = mistakes/len(y)
    return error

In [28]:
evaluate_classification_error(tree1,Xtrain,y)

0.0060000000000000001

In [33]:
leavesData1 = map(lambda x: datapath(tree1,x), Xtrain)
print leavesData1

[7, 5, 7, 4, 5, 7, 7, 4, 7, 7, 5, 5, 7, 5, 7, 5, 5, 4, 4, 7, 5, 4, 7, 5, 7, 7, 4, 5, 7, 7, 5, 5, 7, 7, 5, 7, 4, 4, 7, 5, 5, 7, 4, 7, 5, 7, 5, 5, 5, 7, 4, 7, 5, 7, 5, 5, 5, 5, 5, 7, 7, 5, 5, 7, 4, 7, 7, 7, 7, 7, 7, 7, 5, 7, 5, 7, 7, 5, 4, 7, 7, 4, 7, 5, 5, 5, 5, 7, 5, 5, 5, 4, 5, 7, 7, 4, 5, 7, 4, 7, 4, 7, 4, 5, 5, 7, 4, 4, 7, 5, 7, 5, 7, 4, 4, 5, 7, 4, 7, 5, 5, 4, 6, 5, 5, 5, 4, 7, 5, 7, 5, 7, 7, 7, 7, 7, 7, 4, 7, 4, 7, 7, 7, 7, 5, 5, 7, 5, 5, 7, 7, 5, 4, 5, 5, 4, 7, 5, 4, 5, 7, 5, 5, 7, 7, 5, 7, 7, 7, 5, 7, 7, 7, 7, 4, 5, 7, 7, 5, 5, 5, 5, 7, 5, 5, 7, 7, 5, 5, 4, 5, 7, 7, 5, 5, 4, 7, 5, 5, 7, 7, 7, 4, 7, 5, 5, 4, 5, 7, 7, 4, 5, 4, 5, 5, 5, 4, 5, 5, 5, 7, 5, 7, 7, 5, 5, 7, 5, 6, 7, 7, 7, 4, 5, 4, 5, 7, 7, 7, 4, 7, 7, 5, 7, 5, 5, 7, 7, 7, 5, 7, 5, 7, 4, 5, 4, 5, 4, 7, 7, 7, 7, 7, 7, 7, 7, 5, 5, 5, 7, 5, 5, 7, 5, 4, 5, 5, 4, 7, 5, 5, 7, 7, 7, 7, 7, 7, 7, 5, 5, 5, 4, 7, 4, 5, 7, 5, 5, 5, 5, 7, 4, 5, 5, 5, 4, 7, 7, 5, 7, 5, 5, 7, 7, 7, 7, 7, 5, 7, 4, 5, 7, 7, 5, 7, 7, 7, 7, 7, 4, 5, 4, 7, 

## Expansion/Reduction functions for the transfer forest

#### Local Expansion/Reduction of the tree

In [108]:
def expansion_reduction(tree,XT1,yT1,XT2,yT2,max_depth=2,C=2):

    #Tree = tree #a copy of the tree
    
    #finding the leaf where each target datapoint ends up
    leavesData1 = map(lambda x: datapath(tree,x), XT1)
    leavesData2 = map(lambda x: datapath(tree,x), XT2)
            
    Uleaves1 = np.unique(leavesData1)  #the path to each leaf followed by data1
    Uleaves2 = np.unique(leavesData2)  #the path to each leaf followed by data2
    Uleaves = list(set(Uleaves1) & set(Uleaves2)) #leaves reached by both data1 and data2
            
    #expanding each leaf on the 1st bootstrap replica of target data
    for i in Uleaves:
        ind_data1 = leavesData1==i #indices of datapoints for each leaf
        #set the current depth of subtree to 1 to prevent inferring the number of classes from the code
        Exp_tree = decision_tree_create(XT1[ind_data1,:],yT1[ind_data1],XT1.shape[1],current_depth=1,max_depth=max_depth,C=C)

        #Is this a good expansion?: computes classification error at each leaf for Data T2
        ind_data2 = leavesData2==i
        Err_leavesT2 = intermediate_node_num_mistakes(yT2[ind_data2])/len(yT2[ind_data2])

        #error at the current subtree on Data T2
        Err_subtreeT2 = evaluate_classification_error(Exp_tree, XT2[ind_data2], yT2[ind_data2])
        
        #comparing the error of the subtree with that at the leaf node of the original tree
        if Err_subtreeT2 < Err_leavesT2:
            tree = mergetrees(tree,i,Exp_tree)
            print 'merging successful!'
        else:
            print 'no merging: discard subtree'
    
    return tree

In [83]:
def mergetrees(tree1,leafnr,tree2):
    leafnrbin = bin(leafnr)[3:]  #path is from the 4th element of the binary on: 0 = go left, 1 = go right
    path = ''
    for i in range(len(leafnrbin)):
        if leafnrbin[i] == '0':
            path=path+str("['left']")
        else:
            path=path+str("['right']") 
        print(path)
    exec ('tree1'+path+"['prediction']"+'=None')
    exec ('tree1'+path+"['is_leaf']"+'=False')
    exec ('tree1'+path+"['left']"+"=tree2['left']")
    exec ('tree1'+path+"['right']"+"=tree2['right']")
    exec ('tree1'+path+"['splitting_feature']"+"=tree2['splitting_feature']")
    exec ('tree1'+path+"['threshold']"+"=tree2['threshold']")
    exec ('del(tree1'+path+"['labels_distribution'])")
    
#    print ('tree1'+path+"['prediction']"+'=None')
#    print ('tree1'+path+"['is_leaf']"+'=False')
#    print ('tree1'+path+"['left']"+"=tree2['left']")
#    print ('tree1'+path+"['right']"+"=tree2['right']")
#    print ('tree1'+path+"['splitting_feature']"+"=tree2['splitting_feature']")
    
    #print('tree1'+path+'=tree2')
    return tree1
    

In [58]:
Xsource, Xtarget, ysource, ytarget = train_test_split(Xtrain, y, test_size=0.7)
XT1, XT2, yT1, yT2 = train_test_split(Xtarget, ytarget, test_size=0.5, random_state=42)
print ysource.shape
print yT1.shape
print yT2.shape

(300,)
(350,)
(350,)


In [142]:
tree1 = decision_tree_create(Xsource,ysource,Xsource.shape[1],max_depth=2,C=3)

--------------------------------------------------------------------
Subtree, depth = 0 (300 data points).
Features selected = [1 0]
Split on feature 1. ((160,), (140,)), Threshold = 0.54470130192
--------------------------------------------------------------------
Subtree, depth = 1 (160 data points).
Features selected = [0 1]
Split on feature 0. ((30,), (130,)), Threshold = 0.238050299278
--------------------------------------------------------------------
Subtree, depth = 2 (30 data points).
Features selected = [0 1]
No Mistakes at current node - Stopping.
--------------------------------------------------------------------
Subtree, depth = 2 (130 data points).
Features selected = [0 1]
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 1 (140 data points).
Features selected = [1 0]
Split on feature 1. ((10,), (130,)), Threshold = 0.593831430887
---------------------------------------------------------------

In [143]:
tree1['left']

{'is_leaf': False,
 'labels_distribution': None,
 'left': {'is_leaf': True,
  'labels_distribution': array([ 1.,  0.,  0.]),
  'left': None,
  'prediction': 0,
  'right': None,
  'splitting_feature': None},
 'prediction': None,
 'right': {'is_leaf': True,
  'labels_distribution': array([ 0.01538462,  0.98461538,  0.        ]),
  'left': None,
  'prediction': 1,
  'right': None,
  'splitting_feature': None},
 'splitting_feature': 0,
 'threshold': 0.23805029927820243}

In [144]:
tree1['right']

{'is_leaf': False,
 'labels_distribution': None,
 'left': {'is_leaf': True,
  'labels_distribution': array([ 0.1,  0.2,  0.7]),
  'left': None,
  'prediction': 2,
  'right': None,
  'splitting_feature': None},
 'prediction': None,
 'right': {'is_leaf': True,
  'labels_distribution': array([ 0.,  0.,  1.]),
  'left': None,
  'prediction': 2,
  'right': None,
  'splitting_feature': None},
 'splitting_feature': 1,
 'threshold': 0.59383143088714341}

In [145]:
tree2 = expansion_reduction(tree1,XT1,yT1,XT2,yT2,max_depth=2,C=3)

--------------------------------------------------------------------
Subtree, depth = 1 (46 data points).
Features selected = [0 1]
No Mistakes at current node - Stopping.
no merging: discard subtree
--------------------------------------------------------------------
Subtree, depth = 1 (141 data points).
Features selected = [0 1]
Split on feature 0. ((10,), (131,)), Threshold = 0.301046328297
--------------------------------------------------------------------
Subtree, depth = 2 (10 data points).
Features selected = [0 1]
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (131 data points).
Features selected = [1 0]
No Mistakes at current node - Stopping.
no merging: discard subtree
--------------------------------------------------------------------
Subtree, depth = 1 (19 data points).
Features selected = [0 1]
Split on feature 1. ((10,), (9,)), Threshold = 0.563066280151
-----------------------------------

In [120]:
tree2['right']

{'is_leaf': True,
 'labels_distribution': array([ 0.00714286,  0.01428571,  0.97857143]),
 'left': None,
 'prediction': 2,
 'right': None,
 'splitting_feature': None}

## Printing out a decision stump (To be updated to the numpy version!)

In [119]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print "(leaf, label: %s)" % tree['prediction']
        return None
    split_feature, split_value = split_name.split('.')
    print '                       %s' % name
    print '         |---------------|----------------|'
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '  [{0} == 0]               [{0} == 1]    '.format(split_name)
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree'))

In [120]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


### Exploring the intermediate left subtree

The tree is a recursive dictionary, so we do have access to all the nodes! We can use
* `my_decision_tree['left']` to go left
* `my_decision_tree['right']` to go right

In [121]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


### Exploring the left subtree of the left subtree


In [122]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade.A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.B == 0]               [grade.B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [128]:
print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.D == 0]               [grade.D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)
